In [ ]:
import os
os.system('pip install git+https://github.com/huggingface/transformers.git --upgrade')
os.system('pip install pyyaml==5.1')
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
os.system('pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html')

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
os.system('pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html')

## install PyTesseract
os.system('pip install -q pytesseract')

import gradio as gr
import numpy as np
from transformers import LayoutLMv3Processor, AutoModelForTokenClassification
from datasets import load_dataset
from PIL import ImageDraw, ImageFont
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base")
model = AutoModelForTokenClassification.from_pretrained("ihabram/LayoutLMv3_IstVoices")

# load image example
dataset = load_dataset(r'C:\Users\Habram\Documents\thesis-masters\invoice_generation\istvoices_dataset.py', 
                                  cache_dir=r'C:\Users\Habram\.cache', split="test")
image = dataset[2]["image"]

# define id2label, label2color
labels = dataset.features['ner_tags'].feature.names
id2label = {v: k for v, k in enumerate(labels)}
label2color = {
        'r_name':           'lightcoral',
        'r_street':         'brown',
        'r_housenumber':    'red', 
        'r_zip':            'Salmon',
        'r_city':           'chocolate',
        'r_country':        'Sandybrown',
        'r_vat':            'Sienna',
        's_name':           'olive',      
        's_street':         'yellowgreen',
        's_housenumber':    'lawngreen',
        's_zip':            'palegreen',
        's_city':           'forestgreen',
        's_country':        'limegreen',
        's_vat':            'mediumaquamarine',
        's_bank':           'darkgreen',
        's_bic':            'yellow',
        's_iban':           'teal',
        's_tel':            'beige',
        's_email':          'moccasin',
        'i_number':         'aqua',
        'i_date':           'deepskyblue',
        'i_duedate':        'royalblue',
        'i_amount':         'blue',
        'i_currency':       'orange',
        'other':            'magenta'
    }

def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label

def process_image(image):
    width, height = image.size

    # encode
    encoding = processor(image, truncation=True, return_offsets_mapping=True, return_tensors="pt")
    offset_mapping = encoding.pop('offset_mapping')

    # forward pass
    outputs = model(**encoding)

    # get predictions
    predictions = outputs.logits.argmax(-1).squeeze().tolist()
    token_boxes = encoding.bbox.squeeze().tolist()

    # only keep non-subword predictions
    is_subword = np.array(offset_mapping.squeeze().tolist())[:,0] != 0
    true_predictions = [id2label[pred] for idx, pred in enumerate(predictions) if not is_subword[idx]]
    true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(token_boxes) if not is_subword[idx]]

    # draw predictions over the image
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for prediction, box in zip(true_predictions, true_boxes):
        predicted_label = iob_to_label(prediction).lower()
        draw.rectangle(box, outline=label2color[predicted_label])
        draw.text((box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font)
    
    return image

title = "Interactive demo: LayoutLMv3 istvoices dataset"
description = "Demo for Microsoft's LayoutLMv3, a Transformer for state-of-the-art document image understanding tasks."
article = "LayoutLMv3: Pre-training for Document AI with Unified Text and Image Masking"

css = """.output_image, .input_image {height: 600px !important}"""

iface = gr.Interface(fn=process_image, 
                     inputs=gr.inputs.Image(type="pil"), 
                     outputs=gr.outputs.Image(type="pil", label="annotated image"),
                     title=title,
                     description=description,
                     article=article,
                     css=css)
iface.launch(debug=True, share=True)

Found cached dataset istvoices_dataset (C:/Users/Habram/.cache/istvoices_dataset/default/0.0.0/07bb159fcd7e5fd88962d0d6ca202549a4212b7b1caa7b79cc7108797eb8d1c0)
c:\Users\Habram\Anaconda3\envs\LayoutLMv3\lib\site-packages\gradio\inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\Habram\Anaconda3\envs\LayoutLMv3\lib\site-packages\gradio\inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\Habram\Anaconda3\envs\LayoutLMv3\lib\site-packages\gradio\outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860


c:\Users\Habram\Anaconda3\envs\LayoutLMv3\lib\site-packages\transformers\modeling_utils.py:879: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


: 

: 

: 